## 修正Vega
### 为什么要进行修正
对于**不同到期日，或是不同行权价**的期权，由于IV变化对他们的影响是不一致的，所以不能简单相加/减
> 举个简单的例子，对于一当前Vega中性，由近，远月期权各一份组成的组合，当IV变化后，远月期权受到的影响将会远大于近月端，所以会立马出现Vega敞口。故而简单把Vega相加/减作为风险管理手段是没有什么意义的

### 修正Vega（简易加权）
$$ Modified Vega = \sum_{i=1}^n V_i F_i$$
其中V_{i}是以不同到期日分段的Vega，F为权重
一般是先选择一参考到期日（通常选择**最活跃的到期日**，以下皆以3个月到日的为例），然后对其他月份的头寸，按照一时间因子来调整：$\sqrt{90/TTM}$
例：一个月期权Vega的权重为三个月期权的$\sqrt{90/30}=1.73$倍。一年期权的Vega是三个月的$\sqrt{90/365}=0.5$倍

In [58]:
from pandas_datareader import data
import pandas as pd
import numpy as np
import datetime
from py_vollib_vectorized import get_all_greeks,implied_volatility,vectorized_black_scholes

In [46]:
# 构建一期权组合，都由平值期权组成，到期日分别为一个月，三个月，六个月
# 以三月到期为参考到期日
s = 1000
k = 1000
t1,t2,t3 = 1/12,0.25,0.5
sigma = 0.3

vega1 = get_all_greeks('c',s,k,t1,0,sigma)['vega'][0]
vega2 = -get_all_greeks('c',s,k,t2,0,sigma)['vega'][0]
vega3 = get_all_greeks('c',s,k,t3,0,sigma)['vega'][0]

weighted_vega = np.sqrt(t2/t1) * vega1 + vega2 + np.sqrt(t2/t3) * vega3
print(f'original vega = {vega1 + vega2 + vega3}\nweighted vega = {weighted_vega}\nratio = weighted vega/og vega = {weighted_vega/(vega1 + vega2 + vega3)}')

original vega = 1.9665793573297528
weighted vega = 1.9872530104089123
ratio = weighted vega/og vega = 1.010512493687125


### 协方差分段Vega
#### 远期隐含波动率
两个时刻之间的远期的隐含波动率，表示这两个时刻之间的预期波动率

如果把时间拆分为不重叠的均匀切片，每个日期对应交易的期限
![forwardIV](../note_pic/forwardIV.png)

$$\sigma_{t_{0}, t_{n}}^{2}=\sigma_{t_{0}, t_{1}}^{2}+\sigma_{t_{1}, t_{2}}^{2}+\cdots+\sigma_{t_{n-1}, t_{n}}^{2}$$

其中$\sigma_{0, t_{n}}$即传统意义上，市场对这段时间波动率的报价。如$\sigma_{0, 90}$就是一个90天到期的期权的波动率

但一般来说，会使用不均匀的时间切片：$\sigma_{t_{0}, t_{n}}^{2}=\frac{1}{t_{n}-t_{0}}\left[\left(t_{1}-t_{0}\right) \sigma_{t_{0}, t_{1}}^{2}+\left(t_{2}-t_{1}\right) \sigma_{t_{1}, t_{2}}^{2}+\cdots+\left(t_{n}-t_{n-1}\right) \sigma_{t_{n-1}, t_{n}}^{2}\right]$

其中$\sigma_{t_{n-1}, t_{n}}^{2}$是$t_{n-1}$到$t_n$之间的年华方差，两个时间点之间的差可以是几分钟到几个月不等，比如可以认为$t_3-t_2$是一小时，而$t_4-t_3$是一个月

此时，已知0和$t_1$之间的波动率以及0和$t_2$之间的波动率，就可以推导$t_1$和$t_2$之间的局部波动率：
$$\sigma_{t_{0}, t_{2}}^{2}=\frac{1}{t_{2}-t_{0}}\left[\left(t_{1}-t_{0}\right) \sigma_{t_{0}, t_{1}}^{2}+\left(t_{2}-t_{1}\right) \sigma_{t_{1}, t_{2}}^{2}\right]$$

总结一下，可得到：
$$\sigma_{t_{n}-\alpha, t_{n}}=\sqrt{\frac{\sigma_{t_{0}, t_{n}}^{2}\left(t_{n}-t_{0}\right)-\sigma_{t_{0}, t_{n-\alpha}}^{2}\left(t_{n-\alpha}-t_{0}\right)}{\left(t_{n}-t_{n-\alpha}\right)}}$$

#### 计算方法
1. 建立分段远期波动率矩阵
![forwardIV](../note_pic/截屏2022-08-04%2011.42.07.png)

2. 计算波动率变动10%的Vega的敞口（即波动率从15%变动至16.5%带来的PnL）来建立敞口向量E，然后每个乘以10倍的对应日波动率
![forwardIV](../note_pic/截屏2022-08-04%2011.42.17.png)

3. 建立远期波动率的相关性矩阵M
![forwardIV](../note_pic/截屏2022-08-04%2011.42.27.png)

用E的转置乘以M，再乘以E，得到的数字就是整体波动率变动一个标准差的PnL

In [52]:
IV_30 = implied_volatility.vectorized_implied_volatility(0.0685,2.75,2.75,20/252,0,'c')['IV'][0]
IV_90 = implied_volatility.vectorized_implied_volatility(0.1061,2.75,2.75,55/252,0,'c')['IV'][0]
local_vol_30_90 = np.sqrt((55 * IV_90 * IV_90 - 20 * IV_30 * IV_30)/(55 - 20))
print(f'20天到期的期权隐波：{IV_30}\n55天到期的期权隐波：{IV_90}\n20-55的远期波动率为:{local_vol_30_90}')

20天到期的期权隐波：0.22166832935068564
55天到期的期权隐波：0.2070908456208002
20-55的远期波动率为:0.19828025216518413


In [56]:
3.07*np.sqrt(252)

48.734739149809755

In [57]:
get_all_greeks('c',s,k,t3,0,sigma+0.1)['vega'][0] - get_all_greeks('c',s,k,t3,0,sigma)['vega'][0]

-0.012245537309972576

In [59]:
vectorized_black_scholes('c',s,k,t3,0,sigma+0.1) - vectorized_black_scholes('c',s,k,t3,0,sigma)

,Price
0,27.992889


In [60]:
vectorized_black_scholes('c',s,k,t2,0,sigma+0.1) - vectorized_black_scholes('c',s,k,t2,0,sigma)

,Price
0,19.870386


In [62]:
get_all_greeks('c',s,k,t2,0,sigma)['vega'][0]

1.9891070977056344